# Connect to the server with Windows Authentication

Create a VARIABLE to store the connection string

In [208]:
import pandas as pd
import numpy as np
import pyodbc

conx = pyodbc.connect(
    'DRIVER={SQL Server}; SERVER=LAPTOP-19QKRT53; Database=Survey_Sample_A19; TRUSTED_CONNECTION=yes;')

# Templates for Main Queries

In [209]:
#  Will dynamically columns with Answers by Question (will be replaced afterwards if null )
TemplateForAnswerColumn = 'COALESCE( '\
                          '(SELECT a.Answer_Value '\
                          'FROM	Answer as a '\
                          'WHERE a.UserId=u.UserId AND a.SurveyId= <SURVEY_ID> AND a.QuestionId= <QUESTION_ID>) '\
                          ', -1) As ANS_Q<QUESTION_ID> '

In [210]:
#  Template for Null Values
TemplateForNullColumn = 'NULL As ANS_Q<QUESTION_ID> '


In [211]:
#  Template preparation for union of all users, surveyId (for each) and respective answer
TemplateOuterUnionQuery = 'SELECT u.UserId,  <SURVEY_ID> as SurveyId, <DYNAMIC_QUESTION_ANSWERS> '\
                          'FROM	[User] as u '\
                          'WHERE EXISTS( '\
                          'SELECT * FROM Answer as a WHERE a.UserId=u.UserId AND a.SurveyId = <SURVEY_ID>) '

In [212]:
# check if Question in Survey=>1 , if not ==> 0
CurrentQuestions = 'SELECT SurveyId, QuestionId, InSurvey FROM '\
                   '(SELECT SurveyId, QuestionId, 1 as InSurvey '\
                   'FROM SurveyStructure '\
                   'WHERE SurveyId = @currentSurveyId '\
                   'UNION '\
                   'SELECT @currentSurveyId as SurveyId, Q.QuestionId, 0 as InSurvey '\
                   'FROM Question as Q '\
                   'WHERE NOT EXISTS '\
                   '(SELECT * FROM SurveyStructure as S '\
                   'WHERE S.SurveyId = @currentSurveyId AND S.QuestionId = Q.QuestionId) '\
                   ') as t ORDER BY QuestionId'


##  queries for further loops

In [213]:
 # build SQL Query for all SurveyID in Survey table 
AllSurveys = 'SELECT s.SurveyId FROM Survey AS s ORDER BY SurveyId;'

In [214]:
# execute SQL QUery =AllSurvey and store it into data frame
AllSurveyId = pd.read_sql(AllSurveys, conx) # store into dataframe
print(AllSurveyId)

   SurveyId
0         1
1         2
2         3


In [215]:
# reminder of SQL query to check if question in Survey
print(CurrentQuestions) 

SELECT SurveyId, QuestionId, InSurvey FROM (SELECT SurveyId, QuestionId, 1 as InSurvey FROM SurveyStructure WHERE SurveyId = @currentSurveyId UNION SELECT @currentSurveyId as SurveyId, Q.QuestionId, 0 as InSurvey FROM Question as Q WHERE NOT EXISTS (SELECT * FROM SurveyStructure as S WHERE S.SurveyId = @currentSurveyId AND S.QuestionId = Q.QuestionId) ) as t ORDER BY QuestionId


In [216]:
# prepare SQL query for later, progressevily the query will be build dynamically (Use later) 

finalQueryBuild = []

###  steps to get AllSurveyData

***Step 1***

1. for every iteration of SurveyID, @currentSurveyId will be replaced by actual SurveyID to properly build SQL query for connection do db 
2. store output in df  currentQuestions
3. create variable columnsQueryPart as empty list to fill it progressivelly with QuestionID and matching answer

***Step 2***

1. iteration through rows in df to check in Question in Survey
2. if yes ==> append TemplateForAnswerColumnn with replaced <QUESTION_ID> by current question to columnsQueryPart 
3. else ==> append(TemplateForNullColumnwith replaced <QUESTION_ID> by current question to columnsQueryPart 

***Step 3***

1. to columnsQueryPart list add comma to separate columns

***Step 4***

1. Replace <DYNAMIC_QUESTION_ANSWERS> from TemplateOuterUnionQuery with columnsQueryPartJoin (the one with commas)
2. Replace <'SurveyID> with actual current SurveyId    

***Step 5***
1. Build the Final SQL Query to extract all Survey Data



In [217]:
# Works !!!!!
# Step 1
for surveyid in AllSurveyId['SurveyId']:
    currentQuestionsInSurvey = CurrentQuestions.replace('@currentSurveyId', str(surveyid)) # replace SurveyID by @SurveyID from loop
    currentQuestions = pd.read_sql(currentQuestionsInSurvey, conx) # return in df with SurveyID, Qt, InSurvey(0 or 1)
    columnsQueryPart = [] # use later to dynamically create answers value (or NULL) with matching Qt AnswerToQt
# Step 2
    for index, row in currentQuestions.iterrows():  # Iterates over DataFrame rows as (index, Series) pairs
        if row['InSurvey'] == 1: # if question not in Survey
            # append => add a row with Null value with corresponding AnswerToQt (ex. Ans_Q3)
            columnsQueryPart.append(TemplateForAnswerColumn.replace('<QUESTION_ID>', str(row['QuestionId'])))
            
        else:
            columnsQueryPart.append(TemplateForNullColumn.replace('<QUESTION_ID>', str(row['QuestionId'])))
            # Output here above : NULL As ANS_Q1, 'NULL As ANS_Q2 ', 'NULL As ANS_Q3 ', 'NULL As ANS_Q4
# Step 3
    columnsQueryPartJoin = ', '.join(columnsQueryPart)  #  add a comma between columns
# Step 4
    currentUnionQueryBlock = TemplateOuterUnionQuery.replace('<DYNAMIC_QUESTION_ANSWERS>', columnsQueryPartJoin)
    #here above: replace parameter by actual Answer to respectiv question (ANS_Q_ID)
    currentUnionQueryBlock = currentUnionQueryBlock.replace('<SURVEY_ID>', str(surveyid))
# Step 5
    #here above: replace parameter for SurveyID with actual SurveyID
    finalQueryBuild.append(currentUnionQueryBlock)

finalQuery = 'UNION '.join(finalQueryBuild)
AllSurveyData = pd.read_sql(finalQuery, conx)


print(AllSurveyData.head())



   UserId  SurveyId  ANS_Q1  ANS_Q2  ANS_Q3 ANS_Q4
0      42         1     3.0    -1.0     NaN   None
1     296         1     5.0    -1.0     NaN   None
2    1793         1    -1.0     6.0     NaN   None
3    2411         1    -1.0     1.0     NaN   None
4    2634         1     3.0    -1.0     NaN   None


###  turning into function : getAllSurveyData

In [218]:
def getAllSurveyData():
    #  iterate through SurveyId
        for surveyid in AllSurveyId['SurveyId']:  
            # use later to dynamically create answers value (or NULL) with matching Qt AnswerToQt
            columnsQueryPart = [] 
             # replace @SurveyID  by SurveyID from iteration
            currentQuestionsInSurvey = CurrentQuestions.replace('@currentSurveyId', str(surveyid))
            # Store in DF currentQuestions with replaced SurveyID and stored in df 
            currentQuestions = pd.read_sql(currentQuestionsInSurvey, conx)  
            
           # iterate through rows of CurentQuestions_df
            for index, row in currentQuestions.iterrows():  
                # if Question IS part of SurveyID then TemplateforAnswerColumns
                if row['InSurvey'] == 1: 
                    # append TemplateForAnswerColumn (with replaced Questions) to further SQL query
                    columnsQueryPart.append(TemplateForAnswerColumn.replace('<QUESTION_ID>', str(row['QuestionId'])))
                else:
                    # if Question NOT part of SurveyID then append TemplateforNullColumn (with replaced Questions)
                    columnsQueryPart.append(TemplateForNullColumn.replace('<QUESTION_ID>', str(row['QuestionId'])))  
            #  add a comma between columns            
            columnsQueryPartJoin = ', '.join(columnsQueryPart) 
             #  replace Dynamic_QUestion with actual question
            currentUnionQueryBlock = TemplateOuterUnionQuery.replace('<DYNAMIC_QUESTION_ANSWERS>', columnsQueryPartJoin)
            #  replace <Survey_ID> n with actual current SurveyId
            currentUnionQueryBlock = currentUnionQueryBlock.replace('<SURVEY_ID>', str(surveyid))
             # adding currentUnionQueryBlock that to the end of the list finalQueryBuild
            finalQueryBuild.append(currentUnionQueryBlock)
            # add union to join blocks
            AllSurveyData_SQL_Query = 'UNION '.join(finalQueryBuild)
            AllSurveyData = pd.read_sql(AllSurveyData_SQL_Query, conx)
            return AllSurveyData
        print(AllSurveyData.head())



### Check the function  getAllSurveyData

In [219]:
getAllSurveyData()

UserId  SurveyId  ANS_Q1  ANS_Q2  ANS_Q3 ANS_Q4
0          42         1     3.0    -1.0     NaN   None
1         296         1     5.0    -1.0     NaN   None
2        1358         2     NaN    -1.0     9.0   None
3        1387         3     7.0     NaN     NaN   None
4        1548         2     NaN    -1.0     5.0   None
...       ...       ...     ...     ...     ...    ...
4992  1020927         1    -1.0     4.0     NaN   None
4993  1021179         2     NaN    -1.0     7.0   None
4994  1021182         3     9.0     NaN     NaN   None
4995  1021328         3     5.0     NaN     NaN   None
4996  1021749         3     2.0     NaN     NaN   None

[4997 rows x 6 columns]

In [220]:

# AllSurveyData = pd.read_sql(AllSurveyData_SQL_Query, conx)
print(AllSurveyData)

       UserId  SurveyId  ANS_Q1  ANS_Q2  ANS_Q3 ANS_Q4
0          42         1     3.0    -1.0     NaN   None
1         296         1     5.0    -1.0     NaN   None
2        1793         1    -1.0     6.0     NaN   None
3        2411         1    -1.0     1.0     NaN   None
4        2634         1     3.0    -1.0     NaN   None
...       ...       ...     ...     ...     ...    ...
4992  1019491         3     1.0     NaN     NaN   None
4993  1020147         3     3.0     NaN     NaN   None
4994  1021182         3     9.0     NaN     NaN   None
4995  1021328         3     5.0     NaN     NaN   None
4996  1021749         3     2.0     NaN     NaN   None

[4997 rows x 6 columns]


In [221]:
type(AllSurveyData)

pandas.core.frame.DataFrame

## UPDATE AND TRIGGER

### into csv

In [56]:
import os
os.getcwd() # check workind directory to where csv is stored locally

'C:\\Users\\Cristina\\OneDrive\\Documents\\DSTI\\SQL'

In [222]:
# save AllSurveyData df in csv locally
AllSurveyData.to_csv('AllSurveyData.csv') 

In [253]:
# download locally the CSV
# r' requested to deal with specific charact in path

AllSurveyData = pd.read_csv (r'C:\\Users\\Cristina\\OneDrive\\Documents\\DSTI\\SQL\AllSurveyData.csv', index_col=0)
AllSurveyData.head(5)


UserId  SurveyId  ANS_Q1  ANS_Q2  ANS_Q3  ANS_Q4
0      42         1     3.0    -1.0     NaN     NaN
1     296         1     5.0    -1.0     NaN     NaN
2    1793         1    -1.0     6.0     NaN     NaN
3    2411         1    -1.0     1.0     NaN     NaN
4    2634         1     3.0    -1.0     NaN     NaN

#### Getting Survey structure from last AllSurveyData

#####  Step 1 : using melt function

Pandas.melt() unpivots a DataFrame from wide format to long format.
melt() function is useful to massage a DataFrame into a format where one or more columns are identifier variables, while all other columns, considered measured variables, are unpivoted to the row axis, leaving just two non-identifier columns, variable and value

In [255]:
# using column  ANS_Q1, Q2, Q3.. as rows variables and store into a single column Question ID

AllSurveyData_melt=AllSurveyData.loc[:, 'SurveyId':].melt(id_vars='SurveyId', var_name='QuestionId', value_name='Answer')

AllSurveyData_melt.head(10)

SurveyId QuestionId  Answer
0         1     ANS_Q1     3.0
1         1     ANS_Q1     5.0
2         1     ANS_Q1    -1.0
3         1     ANS_Q1    -1.0
4         1     ANS_Q1     3.0
5         1     ANS_Q1    -1.0
6         1     ANS_Q1     7.0
7         1     ANS_Q1    -1.0
8         1     ANS_Q1    -1.0
9         1     ANS_Q1     9.0

In [256]:
# check for distinct question ID
AllSurveyData_melt.QuestionId.unique()

array(['ANS_Q1', 'ANS_Q2', 'ANS_Q3', 'ANS_Q4'], dtype=object)

In [257]:
# check for distinct Survey ID
AllSurveyData_melt.SurveyId.unique()

array([1, 2, 3], dtype=int64)

#####  Step 2 : preparing Question ID column

In [258]:
# converting Question ID column into an integer mathcing with Question ID

# take last character in QuestionID column (make sure to do it only once) so will have numbers in Question Id column
AllSurveyData_melt['QuestionId']= AllSurveyData_melt['QuestionId'].map(lambda x: str(x)[-1])


In [259]:
AllSurveyData_melt.head(10)

SurveyId QuestionId  Answer
0         1          1     3.0
1         1          1     5.0
2         1          1    -1.0
3         1          1    -1.0
4         1          1     3.0
5         1          1    -1.0
6         1          1     7.0
7         1          1    -1.0
8         1          1    -1.0
9         1          1     9.0

In [260]:
AllSurveyData_melt.info() # to check data types

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19988 entries, 0 to 19987
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SurveyId    19988 non-null  int64  
 1   QuestionId  19988 non-null  object 
 2   Answer      8346 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 468.6+ KB


In [261]:
# make sure it is an integer, convert into integer if needed
AllSurveyData_melt[['SurveyId','QuestionId']] = AllSurveyData_melt[['SurveyId','QuestionId']].apply(pd.to_numeric)
AllSurveyData_melt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19988 entries, 0 to 19987
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   SurveyId    19988 non-null  int64  
 1   QuestionId  19988 non-null  int64  
 2   Answer      8346 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 468.6 KB


In [262]:
# check for null values
AllSurveyData_melt.isnull().sum()

SurveyId          0
QuestionId        0
Answer        11642
dtype: int64

In [263]:
#Drop rows which contains any NaN or missing value
AllSurveyData_melt= AllSurveyData_melt.dropna()

In [264]:
# get rid of duplicates
AllSurveyData_melt.drop_duplicates(['SurveyId','QuestionId'], keep='first', inplace=True)
AllSurveyData_melt


SurveyId  QuestionId  Answer
0             1           1     3.0
3349          3           1     7.0
4997          1           2    -1.0
6671          2           2    -1.0
11668         2           3     9.0

In [265]:
# rename df to a more meaningfull name
Last_AllSurveyData_Structure=AllSurveyData_melt.loc[:, 'SurveyId':'QuestionId']

In [275]:
# order by SurveyId
Last_AllSurveyData_Structure=Last_AllSurveyData_Structure.sort_values(by=['SurveyId'])
Last_AllSurveyData_Structure.head(15)


SurveyId  QuestionId
0         1           1
2         1           2
3         2           2
4         2           3
1         3           1

In [276]:
# rest index

Last_AllSurveyData_Structure.reset_index(inplace = True, drop = True)
# drop=True ==> remove own index with default index 
 
Last_AllSurveyData_Structure.head()

SurveyId  QuestionId
0         1           1
1         1           2
2         2           2
3         2           3
4         3           1

#### Survey Structure in SQL

In [277]:
SurveyStructure_Query='SELECT SurveyId, QuestionId  FROM SurveyStructure'
SurveyStructure = pd.read_sql(SurveyStructure_Query, conx) 
SurveyStructure


SurveyId  QuestionId
0         1           1
1         1           2
2         2           2
3         2           3
4         3           1

### Compare to files (Last Survey with Current Survey)

In order to test : do some modification in table SurveyStructure in DB first

In [278]:
(SurveyStructure).equals(Last_AllSurveyData_Structure)

True

In [279]:
if (SurveyStructure).equals(Last_AllSurveyData_Structure):
    print("There is no updates")
else :
    getAllSurveyData()
    print('Survey is updated now')

There is no updates


### Close connection

In [280]:
csr = conx.cursor()  
csr.close()
del csr

### Hoooray !!! :)